In [1]:
import os
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Input, Activation, Add, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Convolution3D, MaxPooling3D, GlobalAveragePooling3D, Lambda, UpSampling3D, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from keras.losses import mse
%matplotlib inline

C:\Users\tfukushima1\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tfukushima1\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tfukushima1\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tfukushima1\AppData\Roami

In [82]:
def Encoder(latent_dim = 100, channel = 1, frame = 3, height = 100, width = 100):
    input_data = Input(shape=(frame, height, width, channel))

    x = Convolution3D(64, (3,3,3),  kernel_initializer='he_normal', padding='same', activation="relu")(input_data)
    x = MaxPooling3D((1,2,2), padding="same")(x)        
    
    x = Convolution3D(128, (3,3,3),  kernel_initializer='he_normal', padding='same', activation="relu")(x)
    x = MaxPooling3D((3,2,2), padding="same")(x)        
    shape = K.int_shape(x)
    print(shape)
    x = Flatten()(x)
    z_mean = Dense(latent_dim, activation='linear')(x)
    z_log_var = Dense(latent_dim, activation='linear')(x)
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    return Model(input_data, [z_mean, z_log_var, z])

In [83]:
encoder = Encoder(channel=1, frame=3, height=100, width=100)

(None, 1, 25, 25, 128)


In [84]:
encoder.summary()

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, 3, 100, 100, 0                                            
__________________________________________________________________________________________________
conv3d_46 (Conv3D)              (None, 3, 100, 100,  1792        input_24[0][0]                   
__________________________________________________________________________________________________
max_pooling3d_26 (MaxPooling3D) (None, 3, 50, 50, 64 0           conv3d_46[0][0]                  
__________________________________________________________________________________________________
conv3d_47 (Conv3D)              (None, 3, 50, 50, 12 221312      max_pooling3d_26[0][0]           
___________________________________________________________________________________________

In [87]:
def Decoder(latent_dim = 100, channel = 1, frame = 3, height = 100, width = 100):
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = Dense(1 * 25 * 25 * 128, activation='relu')(latent_inputs)
    x = Reshape((1, 25, 25, 128))(x)

    x = UpSampling3D((3, 2, 2))(x)
    x = Convolution3D(128, (3,3,3), activation='relu', padding='same')(x)
    x = UpSampling3D((1, 2, 2))(x)
    x = Convolution3D(64, (3,3,3), activation='relu', padding='same')(x)
    decoded = Convolution3D(1, (3,3,3), activation='relu', padding='same')(x)
    return Model(latent_inputs, decoded)

In [88]:
decoder = Decoder()
decoder.summary()

Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_sampling (InputLayer)      [(None, 100)]             0         
_________________________________________________________________
dense_39 (Dense)             (None, 80000)             8080000   
_________________________________________________________________
reshape_11 (Reshape)         (None, 1, 25, 25, 128)    0         
_________________________________________________________________
up_sampling3d_26 (UpSampling (None, 3, 50, 50, 128)    0         
_________________________________________________________________
conv3d_51 (Conv3D)           (None, 3, 50, 50, 128)    442496    
_________________________________________________________________
up_sampling3d_27 (UpSampling (None, 3, 100, 100, 128)  0         
_________________________________________________________________
conv3d_52 (Conv3D)           (None, 3, 100, 100, 64)   221

In [89]:
def vae_loss(z_mean, z_log_var):
    reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
    
    reconstruction_loss *= 100 * 100
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    
    return vae_loss

In [90]:
# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def Sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
reconstruction_loss *= 100 * 100
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)

In [15]:
is_channels_first = False
if is_channels_first:
  buf = np.load('../ResNet/resnet3d_data_channel_first_updown_and_wipe_frame-3_stride123_gray_100x100_20191016.npz')

else:
  buf = np.load('../ResNet/resnet3d_data_channel_last_updown_and_wipe_frame-3_stride123_gray_100x100_20191016.npz')

X = buf["X"].astype("f2")
Y = buf["Y"]
del buf

Y = keras.utils.to_categorical(Y, 2, dtype="f2").astype("f2")
X_train, X_val,  Y_train,  Y_val = train_test_split(X, Y, train_size=0.8, shuffle=True)
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, train_size=0.8, shuffle=True)

c:\users\tfukushima1\appdata\local\continuum\anaconda3\envs\techacademy\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [27]:
encoder = Encoder(channel=1, frame=3, height=100, width=100)

In [ ]:
decoder = Decoder()

In [94]:
inputs = Input(shape=(3, 100, 100, 1))
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs)

In [98]:
va
vae.add_loss(vae_loss)

In [101]:
adam = keras.optimizers.Adam(lr=0.00001)
#lr_decay = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.0001 + 0.02 * np.math.pow(0.5, 1+epoch), verbose=True)
vae.compile(optimizer=adam)

ValueError: Could not interpret optimizer identifier: <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x000002932C4F2C50>

In [3]:
class VAE:
    def __init__(self, frame = 3, height = 100, width = 100, channel = 1, latent_dim = 100):
        self.frame = frame
        self.height = height
        self.width = width
        self.channel = channel
        self.latent_dim = latent_dim
        self.input = Input(shape=(self.frame, self.height, self.width, self.channel))
        self.latent_inputs = Input(shape=(self.latent_dim,))        

    def Build_Encoder(self):
        x = Convolution3D(64, (3,3,3),  kernel_initializer='he_normal', padding='same', activation="relu")(self.input)
        x = MaxPooling3D((1,2,2), padding="same")(x)        

        x = Convolution3D(128, (3,3,3),  kernel_initializer='he_normal', padding='same', activation="relu")(x)
        x = MaxPooling3D((3,2,2), padding="same")(x)        
        self.shape = K.int_shape(x)

        x = Flatten()(x)
        z_mean = Dense(self.latent_dim, activation='linear')(x)
        z_log_var = Dense(self.latent_dim, activation='linear')(x)
        z = Lambda(self.Sampling, output_shape=(self.latent_dim,), name='z')([z_mean, z_log_var])
        return Model(self.input, [z_mean, z_log_var, z])
    
    def Build_Decoder(self):
        x = Dense(self.shape[1] * self.shape[2] * self.shape[3] * self.shape[4], activation='relu')(self.latent_inputs)
        x = Reshape((self.shape[1], self.shape[2], self.shape[3], self.shape[4]))(x)

        x = UpSampling3D((3, 2, 2))(x)
        x = Convolution3D(128, (3,3,3), activation='relu', padding='same')(x)
        x = UpSampling3D((1, 2, 2))(x)
        x = Convolution3D(64, (3,3,3), activation='relu', padding='same')(x)
        decoded = Convolution3D(1, (3,3,3), activation='relu', padding='same')(x)
        return Model(self.latent_inputs, decoded)        
    
    def Sampling(self, args):
        """
        reparameterization trick
        instead of sampling from Q(z|X), sample eps = N(0,I)
        z = z_mean + sqrt(var)*eps
        """
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        # by default, random_normal has mean=0 and std=1.0
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    def Build(self):
        self.encoder = self.Build_Encoder()
        self.decoder = self.Build_Decoder()
        z_mean, z_log_var, z = self.encoder(self.input)
        outputs = self.decoder(z)
        self.vae = Model(self.input, outputs)

        reconstruction_loss = mse(K.flatten(self.input), K.flatten(outputs))

        reconstruction_loss *= self.shape[1] * self.shape[2] * self.shape[3]
        kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        vae_loss = K.mean(reconstruction_loss + kl_loss)
        self.vae.add_loss(vae_loss)
        self.vae.compile(optimizer="adam")
        
    def fit(self, X_train, X_test, batch_size, epoch):
        self.vae.fit(X_train, batch_size = batch_size, epochs = epoch, validation_data=(X_test, None))        

In [4]:
vae = VAE()

In [5]:
vae.Build()

W1020 17:44:35.863726 18412 deprecation.py:506] From C:\Users\tfukushima1\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1020 17:44:36.501116 18412 training_utils.py:1101] Output model_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to model_1.


In [6]:
is_channels_first = False
if is_channels_first:
  buf = np.load('../ResNet/resnet3d_data_channel_first_updown_and_wipe_frame-3_stride123_gray_100x100_20191016.npz')

else:
  buf = np.load('../ResNet/resnet3d_data_channel_last_updown_and_wipe_frame-3_stride123_gray_100x100_20191016.npz')

X = buf["X"].astype("f2")
Y = buf["Y"]
del buf

Y = keras.utils.to_categorical(Y, 2, dtype="f2").astype("f2")
X_train, X_val,  Y_train,  Y_val = train_test_split(X, Y, train_size=0.8, shuffle=True)
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, train_size=0.8, shuffle=True)

c:\users\tfukushima1\appdata\local\continuum\anaconda3\envs\techacademy\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
vae.fit(X_train, X_test, batch_size=2, epoch=2)

Train on 1574 samples, validate on 394 samples
Epoch 1/2
  30/1574 [..............................] - ETA: 1:06:23 - loss: 22736260935.8412